In [ ]:
import os

import numpy as np
import pandas as pd

import climin
import climin.stops

import theano
import theano.tensor as T

from breze.learn import sgvb
from breze.learn.base import theanox
from breze.arch.construct.layer.distributions import DiagGauss
from breze.arch.construct.neural import Mlp

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
GPU = theano.config.device.startswith('gpu')
if GPU:
    import os
    os.environ['GNUMPY_IMPLICIT_CONVERSION'] = 'allow'

## Loading Data

In [ ]:
# forlder names for each object to be reconstructed
object_folders = ['cirPrism', 'fish', 'fishSQ', 'hut', 'hutWave', 'triangPrism']

In [ ]:
contact_objects = [np.array(pd.read_csv('data/'+f+'/set01-trial01//contactPoints.csv'))[:, :3] for f in object_folders]
contact = np.concatenate(contact_objects, 0).astype('float32')

taxel_objects = [pd.read_csv('data/'+f+'/set01-trial01//contactTactileRaw.csv') for f in object_folders]
taxel = np.concatenate(taxel_objects, 0).astype('float32')

In [ ]:
# split dataset
ind = np.arange(taxel.shape[0])
#np.random.shuffle(ind)
split = 0.7

In [ ]:
X = taxel[ind[:int(len(ind)*split)]]
TX = taxel[ind[int(len(ind)*split):]]

## Normalisation

In [ ]:
mean = X.mean(0)
X -= mean
std = X.std(0)
X /= std
TX -= mean
TX /= std

## Train Variational Autoencoder

In [ ]:
class MlpDiagConstVarGauss(DiagGauss):
    def __init__(self, inpt, n_inpt, n_hiddens, n_output,
                 hidden_transfers, out_transfer_mean='identity',
                 declare=None, name=None, rng=None):
        self.inpt = inpt
        self.n_inpt = n_inpt
        self.n_hiddens = n_hiddens
        self.n_output = n_output
        self.hidden_transfers = hidden_transfers
        self.out_transfer_mean = out_transfer_mean
        self.mean_mlp = Mlp(
            self.inpt, self.n_inpt, self.n_hiddens, self.n_output,
            self.hidden_transfers,
            self.out_transfer_mean,
            declare=declare)
        self.std = declare((1, n_output))
        super(MlpDiagConstVarGauss, self).__init__(
            self.mean_mlp.output,
            T.exp(self.std))
            
            
class MlpGaussConstVarVisibleVAEMixin(object):
    def make_gen(self, latent_sample):
        return MlpDiagConstVarGauss(
            latent_sample, self.n_latent,
            self.n_hiddens_gen,
            self.n_inpt,
            self.gen_transfers,
            declare=self.parameters.declare)

class MyVae(sgvb.VariationalAutoEncoder,
                    sgvb.MlpGaussLatentVAEMixin,
                    MlpGaussConstVarVisibleVAEMixin):
    pass

optimizer = 'rmsprop', {'step_rate': 0.001}
batch_size = 200

n_latent = 5

hiddens_recog = [512] * 2
m = MyVae( int(X.shape[1]),
                    hiddens_recog, n_latent, [512] * 2,
                    ['sigmoid'] * 2, ['sigmoid'] * 2,
                    optimizer=optimizer, batch_size= batch_size)

In [ ]:
climin.initialize.randomize_normal(m.parameters.data, 0, 0.1)

In [ ]:
m.optimizer = 'adam'

In [ ]:
max_passes = 1000
max_iter = max_passes * X.shape[0] / batch_size
n_report = X.shape[0] / batch_size

stop = climin.stops.AfterNIterations(max_iter)
pause = climin.stops.ModuloNIterations(n_report)

for i, info in enumerate(m.powerfit((X,), (X[0:1],), stop, pause)):
    print i, info['loss'], info['val_loss']

In [ ]:
f_latents = m.function(['inpt'], m.vae.recog.sample())
f_meanvar = m.function(['inpt'], m.vae.recog.stt)

## Compute latent values and save plots

In [ ]:
for c, t, f in zip(contact_objects, taxel_objects, object_folders):
    plt.figure()
    l=f_meanvar(theanox((t-mean)/std).astype('float32'))
    for i in range(l.shape[1]):
        plt.scatter(c[:, 0], c[:, 1], c=l[:,i])
        plt.axis('equal')
        plt.savefig('data/'+f+'/latent_'+str(i)+'.png')

## Save parameters for prediction with standalone python script

In [ ]:
for i,l in enumerate(m.vae.recog.mlp.layers):
    np.save('data/W' + str(i) + '.npy', m.parameters[l.weights])
    np.save('data/b' + str(i) + '.npy', m.parameters[l.bias])
np.save('data/mean.npy', mean)
np.save('data/std.npy', std)

## Test output from external python script

In [ ]:
__latents = np.genfromtxt('./data/cirPrism/set01-trial01/latent.csv', delimiter=',')
__coord = np.genfromtxt('./data/cirPrism/set01-trial01/contactPoints.csv', delimiter=',')

In [ ]:
plt.scatter(__coord[:, 0], __coord[:, 1], c=__latents[:,1])
plt.axis('equal');